### Disclaimer:
> Parts of this code were created with the help of ChatGPT

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, RDF, URIRef, Literal, XSD

In [2]:
# Load our current Knowledge Graphs including "owl:sameAs" connections
g = Graph()
g.parse('./full_KG.ttl')

<Graph identifier=N9448836f7a284ccea82a11929eab832d (<class 'rdflib.graph.Graph'>)>

In [3]:
# Creation an endpoint to send the sparql queries to
endpoint = "https://dbpedia.org/sparql"
sparql = SPARQLWrapper(endpoint)


In [4]:
# Get all the artist URIs from dbpedia
q = """
PREFIX ex: <http://example.org/>
PREFIX schema: <https://schema.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT DISTINCT ?realArtist
WHERE {
  ?song schema:byArtist ?artist .
  ?artist owl:sameAs ?realArtist .
}
"""

artist_uris = []
result = g.query(q)
for r in result:
    artist_uris.append(str(r[0]))

In [5]:
# For all Aristst, create a new query that is looking for the birthDate and birthPlace
all_results = []
for artist_uri in artist_uris:
    bdpl_query = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbp: <http://dbpedia.org/property/>

    SELECT ?birthdate ?birthplace
    WHERE {{
        <{artist_uri}> dbo:birthDate ?birthdate .
        <{artist_uri}> dbp:birthPlace ?birthplace .
    }}
    """
    
    # Run the query
    sparql.setQuery(bdpl_query)
    sparql.setReturnFormat(JSON)
    all_results.append(sparql.query().convert())

In [6]:
# Some results did not find anything
print(all_results[0]) # found nothing
print(all_results[4]) # found something

{'head': {'link': [], 'vars': ['birthdate', 'birthplace']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
{'head': {'link': [], 'vars': ['birthdate', 'birthplace']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'birthdate': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '1996-07-11'}, 'birthplace': {'type': 'literal', 'xml:lang': 'en', 'value': 'Mississauga, Ontario, Canada'}}]}}


In [7]:
for a, r in zip(artist_uris, all_results):
    if len(r['results']['bindings']) == 0: # if nothing was found, ignore this URI and go to the next
        continue
    
    print(f"{a} was born in {r['results']['bindings'][0]['birthplace']['value']} on {r['results']['bindings'][0]['birthdate']['value']}")

http://dbpedia.org/resource/Adele was born in London, England on 1988-05-05
http://dbpedia.org/resource/Alan_Walker was born in Northampton, England on 1997-08-24
http://dbpedia.org/resource/Alessia_Cara was born in Mississauga, Ontario, Canada on 1996-07-11
http://dbpedia.org/resource/Alesso was born in http://dbpedia.org/resource/Stockholm on 1991-07-07
http://dbpedia.org/resource/Alicia_Keys was born in New York City, U.S. on 1981-01-25
http://dbpedia.org/resource/Ariana_Grande was born in Boca Raton, Florida, U.S. on 1993-06-26
http://dbpedia.org/resource/Austin_Mahone was born in San Antonio, Texas, U.S. on 1996-04-04
http://dbpedia.org/resource/Avicii was born in Stockholm, Sweden on 1989-09-08
http://dbpedia.org/resource/Bebe_Rexha was born in Brooklyn, New York City, U.S. on 1989-08-30
http://dbpedia.org/resource/Birdy_(singer) was born in Lymington, Hampshire, England on 1996-05-15
http://dbpedia.org/resource/Britney_Spears was born in McComb, Mississippi, U.S. on 1981-12-02
h

In [8]:
# Create a new knowledge graph with the new infos
g_result = Graph()
for a, r in zip(artist_uris, all_results):
    if len(r['results']['bindings']) == 0:
        continue
    
    g_result.add((URIRef(a), 
                 URIRef("http://dbpedia.org/ontology/birthDate"), 
                 Literal(r['results']['bindings'][0]['birthdate']['value'], datatype=XSD.date)))
    g_result.add((URIRef(a),
                 URIRef("http://dbpedia.org/ontology/birthPlace"),
                 Literal(r['results']['bindings'][0]['birthplace']['value'], lang='en')))


In [9]:
g_result.serialize('bdpl.ttl', format='ttl')

<Graph identifier=N4e1f2ef02d0f438ab9bc9da81b38edc9 (<class 'rdflib.graph.Graph'>)>